In [24]:
%matplotlib inline
%config inlineBackend.figure_format='retina'
from __future__ import absolute_import,print_function, absolute_import, division
import matplotlib as mpl
from  matplotlib import pyplot as plt
import seaborn as snb
import pandas as pd
import numpy as np
import time
import os, sys
from tqdm import tqdm

import missingno as msno
import warnings
# ignore the warnings
warnings.filterwarnings("ignore") 
snb.set_context("poster",font_scale=1.3)
import pandas_profiling
from sklearn.datasets import make_blobs



In [102]:
def sub_groups(dataframe,index_name,subgroupname,preficx='raw_' ):
    save_subgroupname="".join([prefix,subgroupname,".csv.gz"])
    test_df=pd.read_csv(save_subgroupname,compression='gzip',index_col=g_index,encoding='UTF-8')
    if test_df.equals(dataframe):
       print("Both datasets are same")
    else:
       print("Datasets are not same")

In [103]:
def load_subgroup(filename,index=[0]):
    return pd.read_csv(filename,compression="gzip",index_col=index)


In [104]:
cleaned_players=load_subgroup('cleaned_players.csv.gz')

In [105]:
players=load_subgroup('raw_players_.csv_gz')

In [106]:
countries=load_subgroup('raw_countries_.csv_gz')

In [107]:
agg_dyads=load_subgroup('raw_dyads_.csv_gz')

In [108]:
referes=load_subgroup('raw_refree_.csv_gz')

In [109]:
# checking to see if the all the games = sum of victories + defeats+ ties
# the all fnction in pythong works like an and
#You can roughly think of any and all as series of logical or and and operators, respectively.

#any ==>any will return True when at least one of the elements is Truthy. Read about Truth Value Testing.

#all ==>all will return True only when all the elements are Truthy.

#Truth table

#+-----------------------------------------+---------+---------+
#|                                         |   any   |   all   |
#+-----------------------------------------+---------+---------+
#| All Truthy values                       |  True   |  True   |
#+-----------------------------------------+---------+---------+
#| All Falsy values                        |  False  |  False  |
#+-----------------------------------------+---------+---------+
#| One Truthy value (all others are Falsy) |  True   |  False  |
#+-----------------------------------------+---------+---------+
#| One Falsy value (all others are Truthy) |  True   |  False  |
#+-----------------------------------------+---------+---------+
#| Empty Iterable                          |  False  |  True   |
#+-----------------------------------------+---------+---------+

In [110]:
agg_dyads.head(5)

playerShort              position  games  victories  ties  defeats  \
refNum                                                                         
1       lucas-wilchez  Attacking Midfielder      1          0     0        1   
2          john-utaka          Right Winger      1          0     0        1   
3         abdon-prats                   NaN      1          0     1        0   
3          pablo-mari           Center Back      1          1     0        0   
3          ruben-pena      Right Midfielder      1          1     0        0   

        goals  yellowCards  yellowReds  redCards  
refNum                                            
1           0            0           0         0  
2           0            1           0         0  
3           0            1           0         0  
3           0            0           0         0  
3           0            0           0         0

In [111]:
all(agg_dyads['games']==agg_dyads['victories']+agg_dyads['defeats']+agg_dyads['ties'])

True

In [112]:
agg_dyads.head(10)

playerShort              position  games  victories  ties  \
refNum                                                                     
1            lucas-wilchez  Attacking Midfielder      1          0     0   
2               john-utaka          Right Winger      1          0     0   
3              abdon-prats                   NaN      1          0     1   
3               pablo-mari           Center Back      1          1     0   
3               ruben-pena      Right Midfielder      1          1     0   
4             aaron-hughes           Center Back      1          0     0   
4       aleksandar-kolarov         Left Fullback      1          1     0   
4         alexander-tettey  Defensive Midfielder      1          0     0   
4        anders-lindegaard            Goalkeeper      1          0     1   
4             andreas-beck        Right Fullback      1          1     0   

        defeats  goals  yellowCards  yellowReds  redCards  
refNum                                                     
1             1      0            0           0         0  
2             1      0            1           0         0  
3             0      0            1           0         0  
3             0      0            0           0         0  
3             0      0            0           0         0  
4             1      0            0           0         0  
4             0      0            0           0         0  
4             1      0            0           0         0  
4             0      0            0           0         0  
4             0      0            0           0         0

In [113]:
agg_dyads['totalRedCards']=agg_dyads['redCards']+agg_dyads['yellowReds']

In [114]:
agg_dyads['totalRedCards'].head(5)

refNum
1    0
2    0
3    0
3    0
3    0
Name: totalRedCards, dtype: int64

In [115]:
agg_dyads.head(5)

playerShort              position  games  victories  ties  defeats  \
refNum                                                                         
1       lucas-wilchez  Attacking Midfielder      1          0     0        1   
2          john-utaka          Right Winger      1          0     0        1   
3         abdon-prats                   NaN      1          0     1        0   
3          pablo-mari           Center Back      1          1     0        0   
3          ruben-pena      Right Midfielder      1          1     0        0   

        goals  yellowCards  yellowReds  redCards  totalRedCards  
refNum                                                           
1           0            0           0         0              0  
2           0            1           0         0              0  
3           0            1           0         0              0  
3           0            0           0         0              0  
3           0            0           0         0              0

In [116]:
agg_dyads.rename(columns={'redCards':'strictredCards'},inplace=True)

In [40]:
agg_dyads.head(5)

playerShort              position  games  victories  ties  defeats  \
refNum                                                                         
1       lucas-wilchez  Attacking Midfielder      1          0     0        1   
2          john-utaka          Right Winger      1          0     0        1   
3         abdon-prats                   NaN      1          0     1        0   
3          pablo-mari           Center Back      1          1     0        0   
3          ruben-pena      Right Midfielder      1          1     0        0   

        goals  yellowCards  yellowReds  strictredCards  totalRedCards  
refNum                                                                 
1           0            0           0               0              0  
2           0            1           0               0              0  
3           0            1           0               0              0  
3           0            0           0               0              0  
3           0            0           0               0              0

In [120]:
agg_dyads.index

Int64Index([   1,    2,    3,    3,    3,    4,    4,    4,    4,    4,
            ...
            3147, 3147, 3147, 3147, 3147, 3147, 3147, 3147, 3147, 3147],
           dtype='int64', name='refNum', length=146028)

In [121]:
cleaned_players.index

Index(['aaron-hughes', 'aaron-hunt', 'aaron-lennon', 'aaron-ramsey',
       'abdelhamid-el-kaoutari', 'abdou-traore_2', 'abdoulaye-diallo_2',
       'abdoulaye-keita_2', 'abdoulwhaid-sissoko', 'abdul-rahman-baba',
       ...
       'ze-castro', 'zhi-gin-lam', 'zlatan-alomerovic', 'zlatan-ibrahimovic',
       'zlatko-junuzovic', 'zoltan-gera', 'zoltan-stieber', 'zoumana-camara',
       'zubikarai', 'zurutuza'],
      dtype='object', name='playerShort', length=1585)

In [122]:
cleaned_players.index.name

'playerShort'

In [123]:
agg_dyads.index.name

'refNum'

In [124]:
agg_dyads.reset_index(inplace=True)

In [125]:
agg_dyads.set_index('playerShort',inplace=True)

In [126]:
agg_dyads.index.name

'playerShort'

### Merge the dyads and cleaned_players(null skintone rating removed dataframe)
#### Revisit and check what is Cleaned_players dataframe. Does it really have all the records that have null skinTone removed

In [129]:
cleaned_players.shape

(1585, 8)

In [127]:
#Merge the agg_dyads and Clean_players on the index columns
cleaned_players.head(5)

height  weight  SkinTone position_agg  heightclass  \
playerShort                                                                  
aaron-hughes             182.0    71.0     0.125      Defense   mid_height   
aaron-hunt               183.0    73.0     0.125     Attacker   mid_height   
aaron-lennon             165.0    63.0     0.250     Midfield   low_height   
aaron-ramsey             178.0    76.0     0.000     Midfield  vlow_height   
abdelhamid-el-kaoutari   180.0    73.0     0.250      Defense  vlow_height   

                       skintoneclass  birth_date  age_years  
playerShort                                                  
aaron-hughes                   white  1979-11-08       39.0  
aaron-hunt                     white  1986-09-04       32.0  
aaron-lennon                   brown  1987-04-16       31.0  
aaron-ramsey                   white  1990-12-26       28.0  
abdelhamid-el-kaoutari         brown  1990-03-17       28.0

In [128]:
agg_dyads.head(5)

refNum              position  games  victories  ties  defeats  \
playerShort                                                                    
lucas-wilchez       1  Attacking Midfielder      1          0     0        1   
john-utaka          2          Right Winger      1          0     0        1   
abdon-prats         3                   NaN      1          0     1        0   
pablo-mari          3           Center Back      1          1     0        0   
ruben-pena          3      Right Midfielder      1          1     0        0   

               goals  yellowCards  yellowReds  strictredCards  totalRedCards  
playerShort                                                                   
lucas-wilchez      0            0           0               0              0  
john-utaka         0            1           0               0              0  
abdon-prats        0            1           0               0              0  
pablo-mari         0            0           0               0              0  
ruben-pena         0            0           0               0              0

In [50]:
#left_index : boolean, default False
#Use the index from the left DataFrame as the join key(s). If it is a MultiIndex, 
#the number of keys in the other DataFrame (either the index or a number of columns) must match the number of levels
#right_index : boolean, default False
#Use the index from the right DataFrame as the join key. Same caveats as left_index

In [130]:
player_dyad_merged=pd.merge(agg_dyads,cleaned_players,left_index=True,right_index=True)

In [131]:
player_dyad_merged.shape,agg_dyads.shape,cleaned_players.shape

((124621, 19), (146028, 11), (1585, 8))

In [137]:
print ('The index of agg_dyads is', agg_dyads.index.name)
print ('The index of cleaned_players is', cleaned_players.index.name)

The index of agg_dyads is playerShort
The index of cleaned_players is playerShort


In [206]:
cleaned_dyads=agg_dyads.reset_index()[agg_dyads.reset_index().playerShort.isin(cleaned_players.index)]
cleaned_dyads.set_index(['refNum','playerShort'],inplace=True)

In [196]:
cleaned_dyads.shape

(124621, 12)

### Dis aggreate the rows. i.e
##### One player in one game can have only one redcard. A redcard means that a player must leave the field immediately and take no further part in the game.
#### So if there are 10 games in a dyad (player-refree pair), then we can split the that dyad in 10 rows, assigning 1 red card (totalredcard=1) , if any , to each one unless the total cards are 0

In [207]:

# checking to see how dyads with more than one games and >1 redcard
cleaned_dyads[(cleaned_dyads['games']>1) & (cleaned_dyads['totalRedCards']>2) ]

position  games  victories  ties  \
refNum playerShort                                                       
88     sergio-ramos             Right Fullback     32         19     6   
137    ricardo-costa               Center Back      6          1     3   
804    sergio-ramos             Right Fullback     18         12     1   
1214   emmanuel-adebayor        Center Forward     23          9     7   
1886   pallardo           Defensive Midfielder      3          1     1   
2449   angel                    Right Fullback     18         10     5   

                          defeats  goals  yellowCards  yellowReds  \
refNum playerShort                                                  
88     sergio-ramos             7      7           14           2   
137    ricardo-costa            2      1            2           3   
804    sergio-ramos             5      4            6           1   
1214   emmanuel-adebayor        7     10            4           1   
1886   pallardo                 1      0            0           3   
2449   angel                    3      0            3           3   

                          strictredCards  totalRedCards  
refNum playerShort                                       
88     sergio-ramos                    1              3  
137    ricardo-costa                   0              3  
804    sergio-ramos                    2              3  
1214   emmanuel-adebayor               2              3  
1886   pallardo                        0              3  
2449   angel                           0              3

In [198]:
len(cleaned_dyads['games'])

124621

In [210]:
t=cleaned_dyads.index.name

In [231]:
player_dyad_merged.head(4)

refNum     position  games  victories  ties  defeats  goals  \
playerShort                                                                 
aaron-hughes       4  Center Back      1          0     0        1      0   
aaron-hughes      66  Center Back      1          1     0        0      0   
aaron-hughes      77  Center Back     26         13     8        5      0   
aaron-hughes     163  Center Back      2          1     1        0      0   

              yellowCards  yellowReds  strictredCards  totalRedCards  height  \
playerShort                                                                    
aaron-hughes            0           0               0              0   182.0   
aaron-hughes            0           0               0              0   182.0   
aaron-hughes            0           0               0              0   182.0   
aaron-hughes            0           0               0              0   182.0   

              weight  SkinTone position_agg heightclass skintoneclass  \
playerShort                                                             
aaron-hughes    71.0     0.125      Defense  mid_height         white   
aaron-hughes    71.0     0.125      Defense  mid_height         white   
aaron-hughes    71.0     0.125      Defense  mid_height         white   
aaron-hughes    71.0     0.125      Defense  mid_height         white   

              birth_date  age_years  
playerShort                          
aaron-hughes  1979-11-08       39.0  
aaron-hughes  1979-11-08       39.0  
aaron-hughes  1979-11-08       39.0  
aaron-hughes  1979-11-08       39.0

In [235]:
player_dyad_merged.head(100)

position  games  victories  ties  \
refNum playerShort                                                           
1      lucas-wilchez          Attacking Midfielder      1          0     0   
2      john-utaka                     Right Winger      1          0     0   
4      aaron-hughes                    Center Back      1          0     0   
       aleksandar-kolarov            Left Fullback      1          1     0   
       alexander-tettey       Defensive Midfielder      1          0     0   
       anders-lindegaard                Goalkeeper      1          0     1   
       andreas-beck                 Right Fullback      1          1     0   
       antonio-rukavina             Right Fullback      2          2     0   
       ashkan-dejagah                  Left Winger      1          1     0   
       benedikt-hoewedes               Center Back      1          1     0   
       chris-baird            Defensive Midfielder      1          0     0   
       chris-brunt                             NaN      1          0     0   
       daniel-schwaab               Right Fullback      1          1     0   
       dennis-aogo                   Left Fullback      1          1     0   
       george-mccartney              Left Fullback      1          0     0   
       gylfi-sigurdsson       Attacking Midfielder      1          0     1   
       ivan-obradovic                Left Fullback      1          1     0   
       jan-moravek            Attacking Midfielder      1          0     1   
       jan-rosenthal          Attacking Midfielder      1          1     0   
       jonny-evans                     Center Back      1          0     0   
       kyriakos-papadopoulos           Center Back      1          0     1   
       marko-marin            Attacking Midfielder      1          1     0   
       mats-hummels                    Center Back      1          1     0   
       mesut-oezil            Attacking Midfielder      1          1     0   
       milorad-pekovic        Defensive Midfielder      1          1     0   
       nemanja-vidic                   Center Back      2          2     0   
       neven-subotic                   Center Back      2          2     0   
       patrick-ebert               Left Midfielder      1          1     0   
       peter-gulacsi                    Goalkeeper      1          1     0   
       steven-davis_2         Defensive Midfielder      1          0     0   
...                                            ...    ...        ...   ...   
7      jeremy-pied                             NaN      2          1     0   
       jonas-martin                            NaN      1          0     0   
       jonathan-tinhan        Defensive Midfielder      3          0     2   
       jose-saez              Defensive Midfielder      1          1     0   
       julien-sable           Defensive Midfielder      1          1     0   
       kassim-abdallah                         NaN      4          1     3   
       kevin-olimpa                            NaN      1          1     0   
       kossi-agassa                            NaN      3          1     2   
       laurent-koscielny               Center Back      2          1     0   
       leyti-ndiaye                            NaN      3          2     0   
       lionel-cappone                   Goalkeeper      1          0     1   
       ludovic-giuly                  Right Winger      2          1     0   
       mamadou-bah                             NaN      2          1     1   
       mody-traore                             NaN      4          2     1   
       nicolas-fauvergue            Center Forward      3          1     2   
       nolan-roux                   Center Forward      2          2     0   
       olivier-giroud                          NaN      3          0     1   
       omar-daf                     Right Fullback      2          2     0   
       papiss-demba-cisse           Center Forward      2       

In [236]:

columns=['games','totalRedCards']
counter=0
# create an array having total number of games and assign a value of 0
output= [0 for _ in range(sum(player_dyad_merged['games']))]
# now using iterrows to loop the games in the dataframe
# not iterrow will return an index(playershort) and the row for each iteration

# loop through the dataframe
for index,row in cleaned_dyads.reset_index().iterrows():
    
    n=row['games']
    d=row['totalRedCards']
    ref=row['refNum']
    player=row['playerShort']
  
    #for number of games in the each dyad, loop 
    for _ in range(n):
        row['totalRedCards']=1 if (d-_)>0 else 0
        rowlist=list([ref,player,row['totalRedCards']])
        output[counter]=rowlist
        counter=counter+1

In [262]:
tidy_dyads=pd.DataFrame(output,columns=['refNum','playerShort','redcard']).set_index(['refNum','playerShort'])

In [264]:
#tidy_dyads.reset_index().groupby(['playerShort','refNum']).agg(['count']).index

In [265]:
tidy_dyads.shape

(373067, 1)

In [268]:
tidy_dyads.redcard.sum()

3092

### Now I will filterout the refrees who are in this final dyad

In [287]:
len(tidy_dyads.reset_index().refNum.unique())

2978

In [294]:
unique_ref_from_dyad=tidy_dyads.reset_index().refNum.unique()
unique_ref_from_dyad

array([   1,    2,    4, ..., 3145, 3146, 3147], dtype=int64)

In [298]:
refree_indexs_present_inDyad=referes.reset_index().refNum.isin(unique_ref_from_dyad)

In [299]:
clean_refrees=referes.reset_index()[refree_indexs_present_inDyad]

In [300]:
clean_refrees.shape

(2978, 2)

In [304]:
clean_refrees.set_index('refNum',inplace=True)

In [313]:
clean_refrees

refCountry
refNum            
1                1
2                2
4                4
6                6
7                7
9                8
11               6
12               6
13              10
14              11
15              12
16              13
17              14
18              15
19              16
20              17
21               8
22              15
23              16
24              18
25              19
26              20
27               3
28               3
29               8
30              21
31              22
32              23
33              22
34              21
...            ...
3117            41
3119           108
3120           108
3121           108
3122             5
3123            70
3124            41
3125            41
3126            41
3127            15
3128            52
3129            52
3130            70
3131            70
3132            70
3133            70
3134            58
3135            15
3136            22
3137           101
3138            22
3139            55
3140            89
3141            89
3142            89
3143            51
3144            55
3145            21
3146            51
3147            21

[2978 rows x 1 columns]

### Now I will filterout the countries who are in this final dyad

In [314]:
len(clean_refrees.reset_index().refCountry.unique())

160

In [315]:
unique_country_from_dyad=clean_refrees.reset_index().refCountry.unique()

In [316]:
country_indexes_present_inDyad=countries.reset_index().refCountry.isin(unique_country_from_dyad)

In [322]:
clean_countries=countries.reset_index()[country_indexes_present_inDyad]

In [323]:
clean_countries.shape

(160, 8)

In [325]:
clean_countries.set_index('refCountry',inplace=True)

In [326]:
clean_countries.shape

(160, 7)

#### Now we see that tidy_dyads is the key for all. We will save it now 

In [327]:
tidy_dyads.to_csv('cleaned_dyads.csv.gz',compression='gzip',encoding='UTF-8')

In [328]:
tidy_dyads.shape

(373067, 1)